In [37]:
from __future__ import print_function, division
from builtins import range, input

# filter warnings
import warnings
warnings.simplefilter(action="ignore", category=FutureWarning)

import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
#if export to py file, need remove this
# %matplotlib inline
import cv2
import random
import time
import shutil
import os
import pickle
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.models import Sequential, load_model
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Dense, Activation, Flatten, Dropout
from keras.layers import Embedding, concatenate
from keras.callbacks import EarlyStopping
from keras.layers import Bidirectional
from keras.layers.recurrent import LSTM
from keras_self_attention import SeqSelfAttention

from keras import utils, optimizers, Model
import tensorflow as tf
from imblearn.over_sampling import RandomOverSampler
from keras.applications.vgg16 import VGG16


category = "beauty" # to change
category_index = 3

In [8]:
data_train = pd.read_csv("ndsc-advanced/" + category + "_data_info_train_competition.csv")
file_names = pd.DataFrame(data_train['image_path']).image_path
ext = '.jpg'
data_train["image_path"] = [(s + ext) if s.find(".jpg") == -1 else s for s in data_train['image_path']]
print(data_train.image_path[0:5])
# plt.imshow(cv2.cvtColor(cv2.imread(file_names[0] + ext),cv2.COLOR_BGR2RGB))

0    beauty_image/6b2e9cbb279ac95703348368aa65da09.jpg
1    beauty_image/20450222d857c9571ba8fa23bdedc8c9.jpg
2    beauty_image/6a5962bed605a3dd6604ca3a4278a4f9.jpg
3    beauty_image/56987ae186e8a8e71fcc5a261ca485da.jpg
4    beauty_image/9c6968066ebab57588c2f757a240d8b9.jpg
Name: image_path, dtype: object


In [9]:
data_test = pd.read_csv("ndsc-advanced/" + category + "_data_info_val_competition.csv")
#file_names = pd.DataFrame(data_test['image_path']).image_path
ext = '.jpg'
data_test["image_path"] = [(s + ext) if s.find(".jpg") == -1 else s for s in data_test['image_path']]
print(data_test.image_path[0:5])
#plt.imshow(cv2.cvtColor(cv2.imread(file_names[0] + ext),cv2.COLOR_BGR2RGB))

0    beauty_image/1588591395c5a254bab84042005f2a9f.jpg
1    beauty_image/920985ed9587ea20f58686ea74e20f93.jpg
2    beauty_image/90b40e5710f54352b243fcfb0f5d1d7f.jpg
3    beauty_image/289c668ef3d70e1d929d602d52d5d78a.jpg
4    beauty_image/d5b3e652c5822d2306f4560488ec30c6.jpg
Name: image_path, dtype: object


In [10]:
data_train.columns[3:]
for col in data_train.columns[3:]:
    data_train[col] = data_train[col].fillna(-1).astype('int32')
print(data_train.shape)
print(data_train.dtypes)
data_train.head()

(286583, 8)
itemid              int64
title              object
image_path         object
Benefits            int32
Brand               int32
Colour_group        int32
Product_texture     int32
Skin_type           int32
dtype: object


,itemid,title,image_path,Benefits,Brand,Colour_group,Product_texture,Skin_type
0,307504,nyx sex bomb pallete natural palette,beauty_image/6b2e9cbb279ac95703348368aa65da09.jpg,1,157,-1,-1,-1
1,461203,etude house precious mineral any cushion pearl...,beauty_image/20450222d857c9571ba8fa23bdedc8c9.jpg,-1,73,11,7,-1
2,3592295,milani rose powder blush,beauty_image/6a5962bed605a3dd6604ca3a4278a4f9.jpg,-1,393,20,6,-1
3,4460167,etude house baby sweet sugar powder,beauty_image/56987ae186e8a8e71fcc5a261ca485da.jpg,-1,73,-1,6,-1
4,5853995,bedak revlon color stay aqua mineral make up,beauty_image/9c6968066ebab57588c2f757a240d8b9.jpg,3,47,-1,6,-1


In [11]:
data_test.columns[3:]
for col in data_test.columns[3:]:
    data_test[col] = data_test[col].fillna(-1).astype('int32')
print(data_test.shape)
print(data_test.dtypes)
data_test.head()

(76545, 3)
itemid         int64
title         object
image_path    object
dtype: object


,itemid,title,image_path
0,370855998,flormar 7 white cream bb spf 30 40ml,beauty_image/1588591395c5a254bab84042005f2a9f.jpg
1,637234604,maybelline clear smooth all in one bb cream sp...,beauty_image/920985ed9587ea20f58686ea74e20f93.jpg
2,690282890,murah innisfree eco natural green tea bb cream...,beauty_image/90b40e5710f54352b243fcfb0f5d1d7f.jpg
3,930913462,loreal white perfect day cream spf 17 pa white...,beauty_image/289c668ef3d70e1d929d602d52d5d78a.jpg
4,1039280071,hada labo cc cream ultimate anti aging spf 35 ...,beauty_image/d5b3e652c5822d2306f4560488ec30c6.jpg


In [12]:
# tackle first category first - to change

df = data_train.iloc[:,[0,1,2,category_index]].copy()
print("before removing null:", df.shape)
df = df[df.iloc[:,3] != -1]
print(df.iloc[:,3].value_counts())
# print(df.iloc[:,3].value_counts().shape)
print("after removing null:", df.shape)
df.iloc[:,3] = df.iloc[:,3].astype('str') #required to convert to str
print(df.dtypes)
df.head()

before removing null: (286583, 4)
1    37955
3    28419
6    26822
4    12577
5     4930
2     2841
0       12
Name: Benefits, dtype: int64
after removing null: (113556, 4)
itemid         int64
title         object
image_path    object
Benefits      object
dtype: object


,itemid,title,image_path,Benefits
0,307504,nyx sex bomb pallete natural palette,beauty_image/6b2e9cbb279ac95703348368aa65da09.jpg,1
4,5853995,bedak revlon color stay aqua mineral make up,beauty_image/9c6968066ebab57588c2f757a240d8b9.jpg,3
5,6208490,dr pure whitening cream,beauty_image/77e6b7e9d5544adbfda6809b2351c4fa.jpg,6
8,9184082,sunprise all proof spf 50,beauty_image/970b32aa659689f371516e5ceb423e25.jpg,6
11,11425757,milani rose powder blush tea,beauty_image/61c6562efdb08df862c57c393d85165e.jpg,1


In [13]:
len_unique_values = len(df.iloc[:,3].unique())
print(len_unique_values)
attribute = df.columns[3]
print("attribute:", attribute)

7
attribute: Benefits


In [14]:
df_X = df.iloc[:,:3]
df_y = list(df.iloc[:,3])
print(len(df_y))
ros = RandomOverSampler(random_state=0)
X_resampled, y_resampled = ros.fit_resample(df_X, df_y)
print(len(y_resampled))
df_resampled = pd.DataFrame(data=X_resampled, columns=df.columns[0:3])
df_resampled[attribute] = y_resampled
print(df_resampled.iloc[:,3].value_counts())
df_resampled.head()

113556
265685
5    37955
1    37955
2    37955
3    37955
4    37955
6    37955
0    37955
Name: Benefits, dtype: int64


,itemid,title,image_path,Benefits
0,307504,nyx sex bomb pallete natural palette,beauty_image/6b2e9cbb279ac95703348368aa65da09.jpg,1
1,5853995,bedak revlon color stay aqua mineral make up,beauty_image/9c6968066ebab57588c2f757a240d8b9.jpg,3
2,6208490,dr pure whitening cream,beauty_image/77e6b7e9d5544adbfda6809b2351c4fa.jpg,6
3,9184082,sunprise all proof spf 50,beauty_image/970b32aa659689f371516e5ceb423e25.jpg,6
4,11425757,milani rose powder blush tea,beauty_image/61c6562efdb08df862c57c393d85165e.jpg,1


In [15]:
# for train data
df_resampled_records = df_resampled.shape[0]

# train (0) or val (1)
random.seed(30)
train_index = random.sample(range(df_resampled_records),  int(df_resampled_records*0.7)) #adjust the proportion
val_index = list(set(range(df_resampled_records)) - set(train_index))
print("len(train_index):", len(train_index), ", ", "len(val_index):", len(val_index))

len(train_index): 185979 ,  len(val_index): 79706


In [16]:
print(df_resampled.shape)
df_resampled.head()

(265685, 4)


,itemid,title,image_path,Benefits
0,307504,nyx sex bomb pallete natural palette,beauty_image/6b2e9cbb279ac95703348368aa65da09.jpg,1
1,5853995,bedak revlon color stay aqua mineral make up,beauty_image/9c6968066ebab57588c2f757a240d8b9.jpg,3
2,6208490,dr pure whitening cream,beauty_image/77e6b7e9d5544adbfda6809b2351c4fa.jpg,6
3,9184082,sunprise all proof spf 50,beauty_image/970b32aa659689f371516e5ceb423e25.jpg,6
4,11425757,milani rose powder blush tea,beauty_image/61c6562efdb08df862c57c393d85165e.jpg,1


In [17]:
# for test data
data_test_records = data_test.shape[0]

In [18]:
print(data_test.shape)
data_test.head()

(76545, 3)


,itemid,title,image_path
0,370855998,flormar 7 white cream bb spf 30 40ml,beauty_image/1588591395c5a254bab84042005f2a9f.jpg
1,637234604,maybelline clear smooth all in one bb cream sp...,beauty_image/920985ed9587ea20f58686ea74e20f93.jpg
2,690282890,murah innisfree eco natural green tea bb cream...,beauty_image/90b40e5710f54352b243fcfb0f5d1d7f.jpg
3,930913462,loreal white perfect day cream spf 17 pa white...,beauty_image/289c668ef3d70e1d929d602d52d5d78a.jpg
4,1039280071,hada labo cc cream ultimate anti aging spf 35 ...,beauty_image/d5b3e652c5822d2306f4560488ec30c6.jpg


In [19]:
df = df_resampled
df.head()

,itemid,title,image_path,Benefits
0,307504,nyx sex bomb pallete natural palette,beauty_image/6b2e9cbb279ac95703348368aa65da09.jpg,1
1,5853995,bedak revlon color stay aqua mineral make up,beauty_image/9c6968066ebab57588c2f757a240d8b9.jpg,3
2,6208490,dr pure whitening cream,beauty_image/77e6b7e9d5544adbfda6809b2351c4fa.jpg,6
3,9184082,sunprise all proof spf 50,beauty_image/970b32aa659689f371516e5ceb423e25.jpg,6
4,11425757,milani rose powder blush tea,beauty_image/61c6562efdb08df862c57c393d85165e.jpg,1


In [20]:
#for train set

docs = df["title"]
docs.head()
print("docs.shape", docs.shape)
# define class labels
labels = np.array(df.iloc[:,3]) #the category we r interested in
print(labels)
# prepare tokenizer
t = Tokenizer()
t.fit_on_texts(docs)
# print("t.word_index", t.word_index)
vocab_size = len(t.word_index) + 1
# print("vocab_size", vocab_size)

docs.shape (265685,)
['1' '3' '6' ... '6' '6' '6']


In [21]:
# integer encode the documents
encoded_docs = t.texts_to_sequences(docs)
print(len(encoded_docs))
# print("min of min:", min(map(min,encoded_docs))) #min=1 so no conflict w padding
# pad documents to a max length of n words
max_length = max(map(len,encoded_docs))
print(max_length)
padded_docs = pad_sequences(encoded_docs, maxlen=max_length, padding='post')
print(padded_docs)

265685
29
[[   60  6196  5802 ...     0     0     0]
 [   23   259    17 ...     0     0     0]
 [  565   247    79 ...     0     0     0]
 ...
 [  354  1929    13 ...     0     0     0]
 [   84 10017    44 ...     0     0     0]
 [  230  2193   331 ...     0     0     0]]


In [22]:
#for test set
docs_test = data_test["title"]
docs_test.head()
print("docs_test.shape", docs_test.shape)

docs_test.shape (76545,)


In [23]:
# integer encode the documents
encoded_docs_test = t.texts_to_sequences(docs_test)
print(len(encoded_docs_test))
max_length_test = max(map(len,encoded_docs_test))
print(max_length_test)
if(max_length_test <= max_length) :
    padded_docs_test = pad_sequences(encoded_docs_test, maxlen=max_length, padding='post')
else :
    padded_docs_test = pad_sequences(encoded_docs_test, maxlen=max_length_test, padding='post')
    padded_docs_test = padded_docs_test[:,:max_length] #trim to train size
print(padded_docs_test)

76545
27
[[1609  447   75 ...    0    0    0]
 [  29  207   71 ...    0    0    0]
 [  38   94 1145 ...    0    0    0]
 ...
 [  44   50 1014 ...    0    0    0]
 [2824    3   28 ...    0    0    0]
 [ 743   14  134 ...    0    0    0]]


## Reading in embedding file, this is only required to run once to generate the pickle file

In [18]:
# start_time = time.time()

# word_embs = dict()
# word_emb = []

# with open("Codes/Beauty_Generic/indo/id.tsv", 'r', errors='ignore') as fd:
#     for row in fd.readlines():
#         line_arr = row.split()
# #         print("line_arr:", line_arr)
#         bracket_exists = "[" in line_arr #bracket before 1st '['
#         if(bracket_exists): #first line
#             line_arr.pop(2)
#         if(bracket_exists and len(word_emb) != 0): #first line n previously there r items
#             word_emb.pop(0) #remove index
# #             print("\t", word_emb[len(word_emb)-1])
#             word_emb[len(word_emb)-1] = word_emb[len(word_emb)-1][:-1] #last emb's ']'
# #             print("\t", word_emb[len(word_emb)-1])
#             word = word_emb[0]
#             word_emb.pop(0)
# #             print("word_emb:", word_emb)
#             word_embs[word] = list(map(float, word_emb))
# #             print("")
#             word_emb = []
# #             print("word_embs:", word_embs)
#         word_emb = word_emb + line_arr

# word_emb.pop(0)
# word_emb[len(word_emb)-1] = word_emb[len(word_emb)-1][:-1] #last emb's ']'
# word = word_emb[0]
# word_emb.pop(0)
# word_embs[word] = list(map(float, word_emb))

# print("time taken", time.time() - start_time)

# with open("indo_word_embs.pickle", "wb") as f:
#     pickle.dump(word_embs, f)

# print("len(word_embs):", len(word_embs))

In [24]:
with open("indo_word_embs.pickle", "rb") as f:
    word_embs = pickle.load(f)
len(word_embs["young"])

300

In [25]:
embedding_size = 300 #300 means use all
word_embs1 = dict()
for key, value in word_embs.items():
    word_embs1[key] = value[0:embedding_size]
len(word_embs1["young"])

300

In [26]:
embedding_matrix = np.zeros((vocab_size, embedding_size))
for word, i in t.word_index.items():
    embedding_vector = word_embs1.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [27]:
#len_unique_values cannot represent the number of categories, cos they skipped some categories
output_categories = utils.np_utils.to_categorical(labels[train_index,]).shape[1]
print(output_categories)

7


# CNN Model

In [28]:
from keras.layers import Bidirectional, GlobalMaxPool1D
from keras.layers.core import Dense, Dropout
from keras.layers.recurrent import LSTM
from keras.layers import Conv1D

model_cnn = Sequential()
model_cnn.add(Embedding(vocab_size, embedding_size, input_length=max_length))
model_cnn.add(Conv1D(64, kernel_size=3, padding= 'same', activation='relu', input_shape=(None, 28, 300)))
model_cnn.add(Dropout(0.5))
model_cnn.add(Dense(32))
model_cnn.add(Dropout(0.25))
model_cnn.add(Flatten())
model_cnn.add(Dense(output_categories, activation='softmax'))
# compile the model
model_cnn.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])
# summarize the model
print(model_cnn.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 29, 300)           5215500   
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 29, 64)            57664     
_________________________________________________________________
dropout_1 (Dropout)          (None, 29, 64)            0         
_________________________________________________________________
dense_1 (Dense)              (None, 29, 32)            2080      
_________________________________________________________________
dropout_2 (Dropout)          (None, 29, 32)            0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 928)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 7)                 6503      
Total para

In [ ]:
start_time = time.time()
# early stopping
es = EarlyStopping(monitor='val_acc', min_delta=0.001, mode='max', verbose=1, patience=2)
# fit the model
model_cnn.fit(padded_docs[train_index,], utils.np_utils.to_categorical(labels[train_index,]), epochs=15, 
          validation_split=0.33, verbose=1, callbacks=[es])
# evaluate the model
loss, accuracy = model_cnn.evaluate(padded_docs[val_index,], utils.np_utils.to_categorical(labels[val_index,]), verbose=1)
print('Accuracy: %f' % (accuracy*100))
print("time taken", time.time() - start_time)

In [ ]:
 model_cnn.save(category + "_" + attribute +".h5")

# LSTM Model

In [24]:
from keras.layers import Bidirectional, GlobalMaxPool1D
from keras.layers.recurrent import LSTM
from keras.layers import Conv1D

model = Sequential()
model.add(Embedding(vocab_size, embedding_size, input_length=max_length))
model.add(LSTM(64))
model.add(Dense(output_categories, activation='softmax'))
 
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 29, 300)           5215500   
_________________________________________________________________
lstm_1 (LSTM)                (None, 64)                93440     
_________________________________________________________________
dense_1 (Dense)              (None, 7)                 455       
Total params: 5,309,395
Trainable params: 5,309,395
Non-trainable params: 0
_________________________________________________________________
None


In [48]:
print(padded_docs.shape)
print(labels.shape)

(265685, 29)
(265685,)


In [ ]:
start_time = time.time()
# early stopping
es = EarlyStopping(monitor='val_acc', min_delta=0.001, mode='max', verbose=1, patience=2)
# fit the model
model.fit(padded_docs[train_index,], utils.np_utils.to_categorical(labels[train_index,]), epochs=15, 
          validation_split=0.33, verbose=1, callbacks=[es])
# evaluate the model
loss, accuracy = model.evaluate(padded_docs[val_index,], utils.np_utils.to_categorical(labels[val_index,]), verbose=1)
print('Accuracy: %f' % (accuracy*100))
print("time taken", time.time() - start_time)

Train on 124605 samples, validate on 61374 samples
Epoch 1/15
124605/124605 [==============================] - 441s 4ms/step - loss: 0.4014 - acc: 0.8466 - val_loss: 0.2852 - val_acc: 0.8948
Epoch 2/15
124605/124605 [==============================] - 435s 3ms/step - loss: 0.2571 - acc: 0.9034 - val_loss: 0.2532 - val_acc: 0.9036
Epoch 3/15
124605/124605 [==============================] - 432s 3ms/step - loss: 0.2114 - acc: 0.9193 - val_loss: 0.2482 - val_acc: 0.9109
Epoch 4/15
124605/124605 [==============================] - 431s 3ms/step - loss: 0.1809 - acc: 0.9308 - val_loss: 0.2338 - val_acc: 0.9173
Epoch 5/15
124605/124605 [==============================] - 429s 3ms/step - loss: 0.1600 - acc: 0.9389 - val_loss: 0.2412 - val_acc: 0.9223
Epoch 6/15
124605/124605 [==============================] - 424s 3ms/step - loss: 0.1420 - acc: 0.9452 - val_loss: 0.2436 - val_acc: 0.9258
Epoch 7/15
124605/124605 [==============================] - 429s 3ms/step - loss: 0.1278 - acc: 0.9506 - val_

In [ ]:
model.save(category + "_" + attribute +".h5")

# Save Predicted Results to CSV

In [ ]:
results = model.predict(padded_docs_test)
results_index = np.argsort(-results, axis=1)[:,:2] 
print(results_index[:10,])

In [ ]:
results_index = np.array(results_index, str)
results_top2 = np.array(list(map(' '.join, results_index))).reshape(-1,1)
print(len(results_top2))
results_top2

In [ ]:
itemid_list = np.array([s + "_" + attribute for s in np.array(data_test["itemid"], str)]).reshape(-1,1)
print(len(itemid_list))
itemid_list

In [ ]:
results_to_export = np.concatenate((itemid_list, results_top2), axis=1)
results_to_export
np.savetxt("results/" + category + "_" + attribute + ".csv", results_to_export, delimiter=",", fmt='%s')